# Armageddon
## A simulation software produced by Popigai-2021

This is a simulation software for events of one spherical asteroid entering Earth atmosphere, it predicts damage zones, caused by the coming asteroid, on Earth surface. The software also produces some graphs on the interested __

### Usage guideline

The following cell contains all the basic parameters needed to start a simulation.

- After keys in all the parameters in the cell, select 'Cell' -> 'Run Cells' in the tool bar to start a parameter check; proceed to next step if the cell outputs 'OK to proceed'; detail report is output in case of failing the parameter check

- Select 'Kernel' -> 'Restart & Run All' in the tool bar would start the simulation, then scroll down to view the outputs; some cells would take some times to produce outputs depend on the parameters given

In [8]:
## Parameter for Earth
# This set the atmospheric density profile in the simulation
#     'exponential' is default; 'tabular' gives a better simulation
atmos_func = 'exponential'

## Parameters for asteroid 
# This is the radius of the incoming asteroid (meter)
radius = 10
# This is the incoming angle of the asteroid (degree)
angle = 20
# This is the incoming velocity of the asteroid (m/s)
velocity = 19e3
# This is the density of the asteroid (kg/m^3)
density = 3000
# This is the strength of the asteroid (Pa)
strength = 1e6
# This is the entry latitude, longitude and bearing of the asteroid (degree)
entry_lat, entry_lon, bearing = 51.2, 0.7, -35.0
# This is the inital altitude of the incoming asteroid (meter)
init_altitude = 100000
# This is the out put time step you want
dt= 0.05

## Parameters for damage levels
# This set which level of damage is shown on a map
#     True indicates the level is shown on the map
levels = {
    'level 1': True,
    'level 2': True,
    'level 3': True,
    'level 4': True
}

## Parameter for damage mapping levels
# This set the basic level for damage mapping
#     'sector' is default, indicates infomation in the mapping is given at postcode sector level
#     'unit' means infomation given at postcode unit level
info_level = 'sector'

# ================== below are parameter checks ==================
varis = [radius, angle, velocity, density,
         strength, entry_lat, entry_lon, bearing, init_altitude, dt]
names = ['radius', 'angle', 'velocity', 'density',
         'strength', 'entry_latitude', 'entry_longitude', 'bearing', 'init_altitude', 'dt']
types = [0, 1, 0, 0, 0, 2, 3, 4, 0, 0]

if atmos_func not in ['exponential', 'tabular']:
    raise ValueError('atmos_func must be \'exponential\' or \'tabular\'')
    
if info_level not in ['sector', 'unit']:
    raise ValueError('info_level must be \'sector\' or \'unit\'')
    
if (type(levels) != dict
    or len(levels) != 4
    or list(levels.keys()) != ['level 1', 'level 2', 'level 3', 'level 4']):
    raise ValueError('only changing value is allowed for levels')
for l in levels.values():
    if type(l) != bool: raise ValueError('only boolean is allowed in levels')
    

def check(var, var_name, var_type):
    if type(var) not in [int, float]: raise ValueError('%s must be a number' % var_name)
        
    min_d, max_d = 0, 0
    if var_type == 1: min_d, max_d = 0, 90
    elif var_type == 2: min_d, max_d = -90, 90
    elif var_type == 3: min_d, max_d = -180, 180
    elif var_type == 4: min_d, max_d = -180, 360

    if var_type == 0:
        if var <= 0: raise ValueError('%s must be a positive number' % var_name)  
    elif var < min_d or var > max_d:
        raise ValueError('%s must be a number between %i and %i' % (var_name, min_d, max_d))

for v, n, t in zip(varis, names, types):
    check(v, n, t)
    
print('OK to proceed')

please set your initial altitude z0(a recommended value is 100000). 100000
please input time step to show up(a recommended value is 0.05). 0.05
OK to proceed


# === CHANGE WITH CAUTION ===

This cell contains advanced parameters for the simulation, changing is allowed ONLY if you know what you are doing and these parameters WILL NOT be checked.

In [9]:
pressure_levels = {
    'level 1': 1e3,
    'level 2': 3.5e3,
    'level 3': 27e3,
    'level 4': 43e3
}

# === DO NOT CHANGE ANYTHING IN ANY CELL BELOW ===

In [10]:
import armageddon as amgd

In [11]:
planet = amgd.Planet(
    atmos_func=atmos_func, atmos_filename=None,
    Cd=1., Ch=0.1, Q=1e7, Cl=0, alpha=0.3, Rp=6371e3,
    g=9.81, H=8000., rho0=1.2
)

result = planet.solve_atmospheric_entry(
    radius=radius, velocity=velocity, density=density, strength=strength, angle=angle,
    init_altitude=init_altitude, dt=dt, radians=False
)

result = planet.calculate_energy(result)
outcome = planet.analyse_outcome(result)

In [12]:
pressures = []
for p, b in zip(pressure_levels.values(), levels.values()):
    if b: pressures.append(p)

blast_lat, blast_lon, damage_rad = amgd.damage_zones(
    outcome, lat=entry_lat, lon=entry_lon,
    bearing=bearing, pressures=pressures
)

In [13]:
locator = amgd.PostcodeLocator(
    postcode_file='./armageddon/resources/full_postcodes.csv',
    census_file='./armageddon/resources/population_by_postcode_sector.csv'
)

sector = True
if info_level == 'unit': sector = False

postcodes = locator.get_postcodes_by_radius((blast_lat, blast_lon), damage_rad, sector=sector)
population = locator.get_population_of_postcode(postcodes, sector=sector)
amgd.plot_results(
    entry_lat, entry_lon,
    blast_lat, blast_lon, damage_rad,
    postcodes, population, sector=sector
)